<a href="https://colab.research.google.com/github/vimigueloli/ICPD/blob/main/interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class Paciente:

  ##declaração da classe
  def __init__(self,sexo,idade,hipertensao,doenca_do_coracao,ja_casou,tipo_de_trabalho,vive_na_cidade,nivel_de_glucose,peso,altura,fumante):
    if sexo == 'masculino':
      self.sexo = 1
    else:
      self.sexo = 0

    self.idade = idade

    if hipertensao == True:
      self.hipertensao = 1
    else:
      self.hipertensao = 0  

    if doenca_do_coracao == True:
      self.doenca_do_coracao = 1
    else:
      self.doenca_do_coracao = 0  

    if ja_casou == True:
      self.ja_casou = 1
    else:
      self.ja_casou = 0 

    if tipo_de_trabalho == 'autonomo':
      self.tipo_de_trabalho = 3
    elif tipo_de_trabalho == 'privado':
      self.tipo_de_trabalho = 2
    elif tipo_de_trabalho == 'governamental':
      self.tipo_de_trabalho = 0  
    elif tipo_de_trabalho == 'criança':
      self.tipo_de_trabalho = 4   
    else:
      self.tipo_de_trabalho = 1 

    if vive_na_cidade == True:
      self.vive_na_cidade = 1
    else:
      self.vive_na_cidade = 0  

    self.nivel_de_glucose = nivel_de_glucose    

    self.imc = peso / altura**2

    if fumante == 'fumante':
      self.fumante = 2
    elif fumante == 'socialmente':
      self.fumante = 0 
    else:
      self.fumante = 1

  def get_data(self):
    paciente = [[self.sexo,self.idade,self.hipertensao,self.doenca_do_coracao,self.ja_casou,self.tipo_de_trabalho,self.vive_na_cidade,self.nivel_de_glucose,self.imc,self.fumante]]
    paciente = pd.DataFrame(paciente,columns=['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'])
    return paciente

In [2]:
## classe de IA unica
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score 
from sklearn.svm import SVC

class ICPD:

  ##declaração da classe
  def __init__(self,amostra=True,tipo=1):
    self.amostra = amostra
    self.tipo = tipo
    self.banco = []
    self.X = []
    self.Y = []
    if tipo==1:
      self.modelo = SVC(probability = True, kernel= 'rbf')
    elif tipo==2:
      self.modelo = SVC(probability = True, kernel= 'linear')
    elif tipo==3:
      self.modelo = SVC(probability = True, kernel= 'sigmoid')
    elif tipo==4:
      self.modelo = SVC(probability = True, kernel= 'poly',degree=2) 
    elif tipo==5:
      self.modelo = KNeighborsClassifier(weights='distance', n_neighbors= 7)
    elif tipo==6:
      self.modelo = MLPClassifier(random_state=2, hidden_layer_sizes = (200,2),learning_rate= 'adaptive')  
    self.certas = []     
    self.proba  = []

  ##metodo que passa o banco de dados para uma variavel pandas e pre processa eles
  def iniciar(self):
    os.environ['KAGGLE_CONFIG_DIR']='/content'
    !kaggle datasets download -d fedesoriano/stroke-prediction-dataset
    with zipfile.ZipFile('/content/stroke-prediction-dataset.zip', 'r') as zip_ref:
      zip_ref.extractall('/content')
    dados= pd.read_csv('healthcare-dataset-stroke-data.csv')
    dados['smoking_status'] = dados['smoking_status'].replace(['Unknown'], 'never smoked')
    dados[['bmi']]=dados[['bmi']].fillna(26)
    classificador = LabelEncoder() 
    dados[['gender']] = classificador.fit_transform(dados[['gender']])
    dados[['ever_married']] = classificador.fit_transform(dados[['ever_married']])
    dados[['work_type']] = classificador.fit_transform(dados[['work_type']])
    dados[['Residence_type']] = classificador.fit_transform(dados[['Residence_type']])
    dados[['smoking_status']] = classificador.fit_transform(dados[['smoking_status']])
    self.banco = dados

  ##metodo que faz o ajuste da amostra via overampling caso 'amostra' == FALSE ou undersampling caso 'amostra' == TRUE  
  def ajuste(self):
    dados = self.banco
    if self.amostra == True:
      saudaveis = dados[dados.stroke == 0].index
      amostra_s = np.random.choice(saudaveis, 250, replace=False)
      doentes = dados[dados.stroke == 1].index
      amostra = dados.loc[amostra_s]
      amostra = amostra.append(dados.loc[doentes])
      dados = amostra
      x= dados[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
      y= dados[['stroke']]
    else:
      x= dados[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
      y= dados[['stroke']]
      smt=SMOTENC(categorical_features=[0, 2], random_state=0)
      x,y = smt.fit_resample(x,y)
      x = pd.DataFrame(x,columns=['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'])
      x['work_type'] = x['work_type'].round(0)      
      x['smoking_status'] = x['smoking_status'].round(0)  
      x['Residence_type'] = x['Residence_type'].round(0)
      x['heart_disease'] = x['heart_disease'].round(0)
      x['ever_married'] = x['ever_married'].round(0)
      x['age'] = x['age'].round(0)
      y = pd.DataFrame(y,columns=['stroke'])
      dados = x
      dados.insert(10, 'stroke', 0, allow_duplicates=False)
      dados[['stroke']] = y
      x = x[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
    self.banco= dados
    self.X= x
    self.Y= y 

  ##metodo que treina a IA e armazena dados para a geração do grafico
  def treino(self):
    x= self.X
    y= self.Y 
    treino_x, teste_x, treino_y, teste_y = train_test_split(x,y)
    self.modelo.fit(treino_x,treino_y)
    respostas_proba = (self.modelo.predict_proba(teste_x))*100
    respostas = self.modelo.predict(teste_x)
    respostas_proba =pd.DataFrame(respostas_proba,columns=['falha','sucesso']) 
    respostas_proba = respostas_proba[['sucesso']]
    respostas_certas = teste_y*100
    self.certas= respostas_certas
    self.proba = respostas_proba

  ##metodo que gera um grafico para exibição
  def grafico(self):
    respostas_certas = self.certas.head(51)
    respostas_proba = self.proba.head(51)
    altura1 = []
    altura2 = []
    altura3 = []
    for eixo in respostas_certas['stroke']:
      altura1.append(eixo)
    for eixo in respostas_proba['sucesso']:
      altura2.append(eixo)
    for altura in altura1:
      altura3.append((altura-100)*-1)
    tamanho = np.arange(51)
    fig,ax = mp.subplots(figsize=(20,12))
    mp.bar(tamanho,altura3,color = '#92e698')
    mp.bar(tamanho,altura1,color = '#e69292')
    mp.plot(tamanho,respostas_proba[['sucesso']],lw=1,marker='o',ms=6,c='#4946c5')
    mp.xlabel('Pacientes')
    mp.ylabel('porcentagem')
    mp.grid()
    mp.show()
    ##mp.savefig("desempenho.png",dpi=300)
  
  ##metodo que recebe os dados do paciente e da um valor em porcentagem da chance dessa pessoa ter derrame de acordo com o banco de dados analizados
  def diagnostico(self,paciente):
    ficha = paciente.get_data()
    chute = self.modelo.predict_proba(ficha) * 100
    chute = pd.DataFrame(chute,columns=['falha','sucesso'])
    chute = chute['sucesso']
    chute = np.array(chute)
    chute = chute[0]
    return chute

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
## classe de comitê
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score 
from sklearn.svm import SVC

class CCPD:

  ##declaração da classe
  def __init__(self):
    os.environ['KAGGLE_CONFIG_DIR']='/content'
    !kaggle datasets download -d fedesoriano/stroke-prediction-dataset
    with zipfile.ZipFile('/content/stroke-prediction-dataset.zip', 'r') as zip_ref:
      zip_ref.extractall('/content')
    dados = pd.read_csv('healthcare-dataset-stroke-data.csv')
    dados1= pd.read_csv('healthcare-dataset-stroke-data.csv')
    dados['smoking_status'] = dados['smoking_status'].replace(['Unknown'], 'never smoked')
    dados[['bmi']]=dados[['bmi']].fillna(26)
    classificador = LabelEncoder() 
    dados[['gender']] = classificador.fit_transform(dados[['gender']])
    dados[['ever_married']] = classificador.fit_transform(dados[['ever_married']])
    dados[['work_type']] = classificador.fit_transform(dados[['work_type']])
    dados[['Residence_type']] = classificador.fit_transform(dados[['Residence_type']])
    dados[['smoking_status']] = classificador.fit_transform(dados[['smoking_status']])
    dados1['smoking_status'] = dados1['smoking_status'].replace(['Unknown'], 'never smoked')
    dados1[['bmi']]=dados[['bmi']].fillna(26)
    classificador = LabelEncoder() 
    dados1[['gender']] = classificador.fit_transform(dados1[['gender']])
    dados1[['ever_married']] = classificador.fit_transform(dados1[['ever_married']])
    dados1[['work_type']] = classificador.fit_transform(dados1[['work_type']])
    dados1[['Residence_type']] = classificador.fit_transform(dados1[['Residence_type']])
    dados1[['smoking_status']] = classificador.fit_transform(dados1[['smoking_status']])
    saudaveis = dados1[dados1.stroke == 0].index
    amostra_s = np.random.choice(saudaveis, 250, replace=False)
    doentes = dados1[dados1.stroke == 1].index
    amostra = dados1.loc[amostra_s]
    amostra = amostra.append(dados1.loc[doentes])
    dados1  = amostra
    x1= dados1[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
    y1= dados1[['stroke']]
    x= dados[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
    y= dados[['stroke']]
    smt=SMOTENC(categorical_features=[0, 2], random_state=0)
    x,y = smt.fit_resample(x,y)
    x = pd.DataFrame(x,columns=['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status'])
    x['work_type'] = x['work_type'].round(0)      
    x['smoking_status'] = x['smoking_status'].round(0)  
    x['Residence_type'] = x['Residence_type'].round(0)
    x['heart_disease'] = x['heart_disease'].round(0)
    x['ever_married'] = x['ever_married'].round(0)
    x['age'] = x['age'].round(0)
    y = pd.DataFrame(y,columns=['stroke'])
    dados = x
    dados.insert(10, 'stroke', 0, allow_duplicates=False)
    dados[['stroke']] = y
    x = x[['gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status']]
    self.modelo   = MLPClassifier(random_state=2, hidden_layer_sizes = (200,2),learning_rate= 'adaptive')
    self.modelo1  = KNeighborsClassifier(weights='distance', n_neighbors= 10)
    self.modelo2  = SVC(probability = True, kernel= 'poly',degree=2)
    treino_x, teste_x, treino_y, teste_y = train_test_split(x,y)
    treino_x1, teste_x1, treino_y1, teste_y1 = train_test_split(x1,y1)
    self.modelo.fit(treino_x,treino_y)
    self.modelo1.fit(treino_x,treino_y)
    self.modelo2.fit(treino_x1,treino_y1)
    respostas_proba0 = (self.modelo.predict_proba(teste_x))*100
    respostas_proba1 = (self.modelo1.predict_proba(teste_x))*100
    respostas_proba2 = (self.modelo2.predict_proba(teste_x))*100
    respostas0 = self.modelo.predict(teste_x)
    respostas1 = self.modelo1.predict(teste_x)
    respostas2 = self.modelo2.predict(teste_x)
    acuracia  = accuracy_score(teste_y,respostas0)*100
    acuracia1 = accuracy_score(teste_y,respostas1)*100
    acuracia2 = accuracy_score(teste_y,respostas2)*100
    respostas_proba0 =pd.DataFrame(respostas_proba0, columns=['falha','sucesso']) 
    respostas_proba0 = respostas_proba0[['sucesso']]
    respostas_proba1 =pd.DataFrame(respostas_proba1,columns=['falha','sucesso']) 
    respostas_proba1 = respostas_proba1[['sucesso']]
    respostas_proba2 =pd.DataFrame(respostas_proba2,columns=['falha','sucesso']) 
    respostas_proba2 = respostas_proba2[['sucesso']]
    lista  = list(respostas_proba0['sucesso'])
    lista1 = list(respostas_proba1['sucesso'])
    lista2 = list(respostas_proba2['sucesso'])
    listas = (lista,lista1,lista2)
    listas = np.array(listas).T
    probabilidade =pd.DataFrame(listas,columns=['sucesso0','sucesso1','sucesso2']) 
    probabilidade['sucesso'] = probabilidade.sum(axis=1)
    probabilidade['sucesso'] = probabilidade['sucesso']/3
    respostas_proba = np.array(probabilidade['sucesso'])
    self.respostas_proba =pd.DataFrame(respostas_proba,columns=['sucesso'])
    respostas = (respostas0,respostas1,respostas2)
    respostas = np.array(respostas).T
    comite =pd.DataFrame(respostas,columns=['chute','chute1','chute2']) 
    comite['soma'] = comite.sum(axis=1)
    respostas = comite['soma']/3
    respostas = respostas.round()
    self.respostas_certas = teste_y*100

  ##metodo que gera um grafico para exibição
  def grafico(self):
    respostas_certas = self.respostas_certas.head(51)
    respostas_proba = self.respostas_proba.head(51)
    altura1 = []
    altura2 = []
    altura3 = []
    for eixo in respostas_certas['stroke']:
      altura1.append(eixo)
    for eixo in respostas_proba['sucesso']:
      altura2.append(eixo)
    for altura in altura1:
      altura3.append((altura-100)*-1)
    tamanho = np.arange(51)
    fig,ax = mp.subplots(figsize=(20,12))
    mp.bar(tamanho,altura3,color = '#92e698')
    mp.bar(tamanho,altura1,color = '#e69292')
    mp.plot(tamanho,respostas_proba[['sucesso']],lw=1,marker='o',ms=6,c='#4946c5')
    mp.xlabel('Pacientes')
    mp.ylabel('porcentagem')
    mp.grid()
    mp.show()
    ##mp.savefig("desempenho.png",dpi=300)
  
  ##metodo que recebe os dados do paciente e da um valor em porcentagem da chance dessa pessoa ter derrame de acordo com o banco de dados analizados
  def diagnostico(self,paciente):
    ficha = paciente.get_data()
    respostas_proba0 = (self.modelo.predict_proba(ficha))*100
    respostas_proba1 = (self.modelo1.predict_proba(ficha))*100
    respostas_proba2 = (self.modelo2.predict_proba(ficha))*100
    respostas_proba0 = pd.DataFrame(respostas_proba0, columns=['falha','sucesso']) 
    respostas_proba0 = respostas_proba0[['sucesso']]
    respostas_proba1 = pd.DataFrame(respostas_proba1,columns=['falha','sucesso']) 
    respostas_proba1 = respostas_proba1[['sucesso']]
    respostas_proba2 = pd.DataFrame(respostas_proba2,columns=['falha','sucesso']) 
    respostas_proba2 = respostas_proba2[['sucesso']]
    lista  = list(respostas_proba0['sucesso'])
    lista1 = list(respostas_proba1['sucesso'])
    lista2 = list(respostas_proba2['sucesso'])
    listas = (lista,lista1,lista2)
    listas = np.array(listas).T
    probabilidade =pd.DataFrame(listas,columns=['sucesso0','sucesso1','sucesso2']) 
    probabilidade['sucesso'] = probabilidade.sum(axis=1)
    probabilidade['sucesso'] = probabilidade['sucesso']/3
    respostas_proba = np.array(probabilidade['sucesso'])
    respostas_proba =pd.DataFrame(respostas_proba,columns=['sucesso'])
    resultado = respostas_proba['sucesso'].head(1)
    resultado = np.array(resultado)
    resultado = resultado[0]
    return resultado

1º prototipo funcional de tela

In [6]:
##tela para coleta de dados

#@title ICPD inteligencia classificadora para derrame






sexo='masculino' #@param ['masculino','feminino']
idade= 0 #@param {type:"slider", min:0, max:115} 
hipertensao= False #@param {type:"boolean"}
doenca_do_coracao= False #@param {type:"boolean"}
ja_casou= False #@param {type:"boolean"}
tipo_de_trabalho= 'nunca trabalhou'#@param ['autonomo', 'privado', 'governamental', 'criança', 'nunca trabalhou']
vive_na_cidade= True #@param {type:"boolean"}
nivel_de_glucose= 79 #@param {type:"slider", min:50, max:170} 
peso= 75.3 #@param {type:"slider", min:1, max:200, step: 0.1} 
altura= 0.9 #@param {type:"slider", min:0.15, max:2.30, step: 0.01}
fumante='nunca fumou' #@param ['nunca fumou','socialmente','fumante']







paciente  = Paciente(sexo,idade,hipertensao,doenca_do_coracao,ja_casou,tipo_de_trabalho,vive_na_cidade,nivel_de_glucose,peso,altura,fumante)

teste de funcionamento de modelos de machine learning configurados

In [ ]:
rede_neural   =  5 #@param {type:"slider", min:0, max:6,step:1}
undersampling =  True #@param {type:"boolean"}

##aplicando a IA de maneira separada
ia_medica = ICPD(amostra = undersampling,tipo=rede_neural)
ia_medica.iniciar()
ia_medica.ajuste()
ia_medica.treino()
print('\n\nEsse é o desempenho da IA que você escolheu:')
ia_medica.grafico()
resultado = ia_medica.diagnostico(paciente)
print("você tem %.2f"% resultado,'% de chance de ter um derrame')
if resultado > 25:
  print('recomendamos você ir a um medico')

Comitê com as IAs com as configurações que apresentaram o melhor desempenho até o momento

In [ ]:
##aplicando IA com comitê

ia_medica = CCPD()
print('\n\n\n\nEsse é o desempenho da IA que você escolheu:')
ia_medica.grafico()
resultado = ia_medica.diagnostico(paciente)
print("você tem %.2f"% resultado,'% de chance de ter um derrame')
if resultado > 25:
  print('recomendamos você ir a um medico')